In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # graphing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Functions for plotting

# Check the survival rate of a dataframe, and plot it in a pie chart
def survival_rate_chart(dataframe):
    dataframe_name = [x for x in globals() if globals()[x] is dataframe][0]
    num_lived = sum(dataframe.Survived)
    num_pssngrs = dataframe.shape[0]
    
    survival_rate = round((num_lived/num_pssngrs)*100,1)
    
    plt.pie([num_lived, num_pssngrs-num_lived], 
            autopct = '%.0f%%',
           labels = ['Lived','Died'])
    plt.title('Dataset= %s' %dataframe_name)
    plt.text(-0.75,-1.25, '# passengers in the dataset = %i' %num_pssngrs)
    
    plt.show()
    
## Test
#survival_rate_chart(df)

def class_comparison_chart(dataframe):
    survival = ('0','1')
    #make a frequency table
    cross_tab = pd.crosstab(index=dataframe["Survived"], 
            columns=dataframe["Pclass"])
    Pclass_counts = {
        '1': np.array([cross_tab.loc[0][1], cross_tab.loc[1][1]]),
        '2': np.array([cross_tab.loc[0][2], cross_tab.loc[1][2]]),
        '3': np.array([cross_tab.loc[0][3], cross_tab.loc[1][3]]),
    }
    width = 0.5  # the width of the bars: can also be len(x) sequence

    #print(Pclass_counts)

    fig, ax = plt.subplots()
    bottom = np.zeros(2)

    for Pclass, Pclass_count in Pclass_counts.items():
        p = ax.bar(survival, Pclass_count, width, label=Pclass, bottom=bottom)
        bottom += Pclass_count

        ax.bar_label(p, label_type='center')

    ax.set_title('Number of survivors by class')
    ax.set_xlabel('Survived')
    ax.set_ylabel('Count')
    ax.legend()

    plt.show()
    
# # Test
# class_comparison_chart(df)
    
def sex_comparison_chart(dataframe):
    survival = ('0','1')
    # make a frequency table
    cross_tab = pd.crosstab(index=dataframe["Survived"], 
            columns=dataframe["Sex"])
    sex_counts = {
        'Female': np.array([cross_tab.loc[0][0], cross_tab.loc[1][0]]),
        'Male': np.array([cross_tab.loc[0][1], cross_tab.loc[1][1]]),
    }
    width = 0.6  # the width of the bars: can also be len(x) sequence


    fig, ax = plt.subplots()
    bottom = np.zeros(2)

    for sex, sex_count in sex_counts.items():
        p = ax.bar(survival, sex_count, width, label=sex, bottom=bottom)
        bottom += sex_count

        ax.bar_label(p, label_type='center')

    ax.set_title('Number of survivors by sex')
    ax.set_xlabel('Survived')
    ax.set_ylabel('Count')
    ax.legend()

    plt.show()
    
# # Test
# sex_comparison_chart(df)

def boarding_comparison_chart(dataframe):
    survival = ('0','1')
    #make a frequency table
    cross_tab = pd.crosstab(index=dataframe["Survived"], 
            columns=dataframe["Embarked"])
    
    #print(cross_tab)
    Place_counts = {
        'C = Cherbourg': np.array([cross_tab.loc[0][0], cross_tab.loc[1][0]]),
        'Q = Queenstown': np.array([cross_tab.loc[0][1], cross_tab.loc[1][1]]),
        'S = Southampton': np.array([cross_tab.loc[0][2], cross_tab.loc[1][2]]),
    }
    width = 0.5  # the width of the bars: can also be len(x) sequence

    #print(Pclass_counts)

    fig, ax = plt.subplots()
    bottom = np.zeros(2)

    for Place, Place_count in Place_counts.items():
        p = ax.bar(survival, Place_count, width, label=Place, bottom=bottom)
        bottom += Place_count

        ax.bar_label(p, label_type='center')

    ax.set_title('Number of survivors by boarding location')
    ax.set_xlabel('Survived')
    ax.set_ylabel('Count')
    ax.legend()

    plt.show()
    
# # Test
# boarding_comparison_chart(df)

# def age_distribution(dataframe):
    

# On the actual Titanic:

Total # people = 2224 

Total # who died = 1502

Actual Survival Rate = 32.46%

# Data Exploration

In [ ]:
# Read the data
df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

df.head()

In [ ]:
# See the overall survival rate 
survival_rate_chart(df)

**Looks like the data is fairly biased, with 38% survival rate (which is close to the actual, 32%). Consider stratified sampling.**

In [ ]:
# See how much data is missing in each column
print('# Missing entries in Training:','\n',df.isnull().sum(), '\n')
print('# Missing entries in Testing:','\n',test_df.isnull().sum())

**Consider ignoring Cabin variable, and imputing Age variable**

In [ ]:
# Visualize the data spread for numerical cols.
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)

survived=df.loc[df['Survived'] == 1]
died = df.loc[df['Survived'] == 0]

ax0.hist([survived.Age.to_numpy(), died.Age.to_numpy()],bins=8, edgecolor="white", stacked=True)
ax0.set_xlabel('Age')

ax1.hist([survived.SibSp.to_numpy(), died.SibSp.to_numpy()], bins=8, edgecolor="white", stacked=True)
ax1.set_xlabel('Number of Siblings/Spouses Aboard')

ax2.hist([survived.Parch.to_numpy(), died.Parch.to_numpy()],bins=6, edgecolor="white", stacked=True)
ax2.set_xlabel('Number of Parents/Children Aboard')

ax3.hist([survived.Fare.to_numpy(), died.Fare.to_numpy()],bins = 25, edgecolor="white", stacked=True)
ax3.set(xlim = (-10, 175))
ax3.set_xlabel('Fare price (excluding outliars)')

fig.tight_layout()
fig.legend(labels = ['Survived','Died'])
plt.show()

# Summary statistics for the numerical cols. 
df[['Age', 'SibSp','Parch','Fare']].describe()

In [ ]:
# Visualize the relationship between categorical values and survival rate

# See the Survival Rate by class
class_comparison_chart(df)

# See the Surivival Rate by sex
sex_comparison_chart(df)

# See the Survival Rate by boarding location
boarding_comparison_chart(df)

**Sex and class will likely be strong indicators of survival**

# Data Preparation

In [ ]:
def missing_age_flag(prepped_data):
    # Flag the entries with missing Age 
    # The reason the data may be missing is because they died and nobody could ask them
    prepped_data['AgeBin_missing'] = prepped_data.Age.isnull()
    prepped_data['AgeBin_missing'] = prepped_data['AgeBin_missing'].astype(int)
    # prepped_data['Embarked_missing'] = prepped_data.Embarked.isnull()
    # print(prepped_data.shape)
    return prepped_data 


def impute_by_class(prepped_data):
    # Impute missing values of Age by class
    frames = []
    # Split into 3 dataframes (one per class)
    # Compute the mean Age and median Fare in each dataframe, and use that to impute missing values
    for i in range(1,4):
        df_class = prepped_data[prepped_data['Pclass'] == i]
        df_class['Age'].fillna(df_class['Age'].mean(),inplace = True)
        df_class['Fare'].fillna(df_class['Fare'].median(),inplace = True)
        #append the data back together
        frames.append(df_class)
    #combine the data frames
    prepped_data = pd.concat(frames).sort_values('PassengerId')
    return prepped_data


def family_structure(prepped_data):
    # Make 'alone' or 'with family' identifier
    prepped_data['FamilySize'] = prepped_data['SibSp'] + prepped_data['Parch']+1

    prepped_data['IsAlone'] = 1 #initialize with 1=yes
    prepped_data['IsAlone'].loc[prepped_data['FamilySize'] > 1] = 0 # now update to 0=no, if family size is greater than 1
    return prepped_data

def one_hot_labels(prepped_data):
    # Change Pclass to one-hot labeling
    for i in range(1,4):
        prepped_data.loc[prepped_data.Pclass == i, str(i)+'_class'] = 1
        prepped_data.loc[prepped_data.Pclass != i, str(i)+'_class'] = 0
        prepped_data[str(i)+'_class'] = prepped_data[str(i)+'_class'].astype(int)

    # One-hot encode the categorical variables
    prepped_data = pd.get_dummies(prepped_data)
#     print(prepped_data.shape)
#     print(prepped_data.dtypes)
    return prepped_data

In [ ]:
def PREPARE_THE_DATA(dataframe):
    prepped_data = dataframe.loc[:,('PassengerId','Survived','Pclass', 'Sex','Age','SibSp','Parch','Fare','Embarked')]
    prepped_data = missing_age_flag(prepped_data) # Adds 'Age_missing'
#     bins = pd.IntervalIndex.from_tuples([(0, 10), (10,20), (20,30), (30,40), (40,50), (50,60), (60,80)])
#     prepped_data['AgeBin'] = pd.cut(prepped_data['Age'], bins)
    prepped_data = impute_by_class(prepped_data)# Fills in missing Age and Fare values by class
    prepped_data = family_structure(prepped_data) # Adds 'IsAlone' and 'FamilySize'
    prepped_data = prepped_data.dropna() # Drops missing Embarked entries
    prepped_data = one_hot_labels(prepped_data) # One-hot labels Class, Sex, Embarked
    # Drop the unnecessary columns
    prepped_data = prepped_data.drop(['Pclass', 'SibSp','Parch', 'Age'],axis=1)
    return prepped_data

# Test
prepped_data = PREPARE_THE_DATA(df)
prepped_data.head

In [ ]:
prepped_data.corr(method='pearson')

Key observations: 
Age missing & class
Age & class
Embarked & class
Survived & class
Survived & sex
SibSp & ParCh



In [ ]:
# Now split df into train (70%) and validation (30%) data sets.

# # Using random sample
# train=df.sample(frac=0.7,random_state=200)
# validation=df.drop(train.index)

# Using stratified sampling
train = prepped_data.groupby('Survived', group_keys=False).apply(lambda x: x.sample(frac=0.7))
validation = prepped_data[~prepped_data.PassengerId.isin(train.PassengerId)]

In [ ]:
# Separate into the data (X) and the target (Y)
train_X = train.drop(['Survived'], axis=1)
train_Y = train.Survived

validation_X = validation.drop(['Survived'], axis=1)
validation_Y = validation.Survived


In [ ]:
# Check survival rate proportions
survival_rate_chart(train)
survival_rate_chart(validation)

Note: Removing the missing values changed the percentages. It decreased the % who died, so the removed data points were likely people that died (makes sense since they weren't able to find them and ask them later on...) **Keeping the missing Age data in could actually be useful.**

In [ ]:
# # Preview data, if desired
# train.head()
# validation.head()

# Decision Tree Model

In [ ]:
# naive tree
from sklearn import tree

tree_model = tree.DecisionTreeClassifier(max_depth=8)
clf = tree_model.fit(train_X, train_Y)

In [ ]:
validation_pred_tree = clf.predict(validation_X)


total_number_wrong_tree = sum(abs(validation_pred_tree - validation_Y)) 

print( 'Total number it got wrong = ', total_number_wrong_tree )
print( 'Accuracy=', round((1-(total_number_wrong_tree / validation.shape[0]))*100, 2), '%' )

In [ ]:
tree.plot_tree(clf)

# Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(max_depth = 10)
forest = forest_model.fit(train_X,train_Y)


In [ ]:
validation_pred_forest = forest.predict(validation_X)

total_number_wrong_forest = sum(abs(validation_pred_forest - validation_Y)) 

print( 'Total number it got wrong = ', total_number_wrong_forest )
print( 'Accuracy=', round((1-(total_number_wrong_forest / validation.shape[0]))*100, 2), '%' )

# Grad Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradboost_model = GradientBoostingClassifier(n_estimators=100, 
                                             learning_rate=1.0, 
                                             max_depth=10)
Gboost = gradboost_model.fit(train_X,train_Y)


In [ ]:
validation_pred_gboost = Gboost.predict(validation_X)

total_number_wrong_gboost = sum(abs(validation_pred_gboost - validation_Y)) 

print( 'Total number it got wrong = ', total_number_wrong_gboost )
print( 'Accuracy=', round((1-(total_number_wrong_gboost / validation.shape[0]))*100, 2), '%' )

# SVM

In [ ]:
from sklearn import svm 

svm_model = svm.SVC()
svmmod = svm_model.fit(train_X,train_Y)

In [ ]:
validation_pred_svm = svmmod.predict(validation_X)

total_number_wrong_svm = sum(abs(validation_pred_svm - validation_Y)) 

print( 'Total number it got wrong = ', total_number_wrong_svm )
print( 'Accuracy=', round((1-(total_number_wrong_svm / validation.shape[0]))*100, 2), '%' )